In [ ]:
import sys
sys.path.append('../')

import numpy as np
import scipy
import tqdm
import utils
import geometry
from phantom import Phantom
from tissue import Tissue
from transducer import Transducer, Focused, Planewave
from transducer_set import TransducerSet
from simulation import SimProperties, Simulation
from sensor import Sensor
from experiment import Experiment
#from reconstruction import Reconstruction, DAS, Compounding 

import matplotlib.pyplot as plt
import plotly.graph_objects as go

import kwave
import glob

phantom = Phantom(source_path = None,
            voxel_dims = (0.1e-3,0.1e-3,0.1e-3), # size of each voxel
            matrix_dims = (400,400,400), # number of voxels along each axis
            baseline = (1500, 1000), # background medium speed of sound and density
            seed = 5678,)

blood = Tissue(name='blood', c=1578, rho=1060, sigma=11.3, scale=0.00001, label=1)
myocardium = Tissue(name='myocardium', c=1561.3, rho=1081, sigma=30, scale=0.0001, label=2)
fat = Tissue(name='fat', c=1450, rho=920, sigma=10, scale=0.0001, label=3)

pos = 0.7

phantom.add_tissue_sphere((pos,-pos,0), 8, blood)
phantom.add_tissue_sphere((pos,0,0), 16, blood)
phantom.add_tissue_sphere((pos,pos,0), 32, blood)

phantom.add_tissue_sphere((-pos,-pos,0), 16, fat)
phantom.add_tissue_sphere((-pos,0,0), 32, fat)
phantom.add_tissue_sphere((-pos,pos,0), 8, fat)

phantom.add_tissue_sphere((0,-pos,0), 32, myocardium)
phantom.add_tissue_sphere((0,0,0), 8, myocardium)
phantom.add_tissue_sphere((0,pos,0), 16, myocardium)

plt.imshow(phantom.get_complete()[0,:,phantom.matrix_dims[2]//4, :], 
           vmin=np.amin(phantom.get_complete()[0]), vmax=np.amax(phantom.get_complete()[0]))
plt.colorbar()

n_transducers = 4 # number of transducers in trajectory

transducers = [Planewave(elements= 32,
                         element_width = 1e-4, # m
                         elevation = 1e-4 * 32, 
                         sweep = np.pi/3, # angles from -60 to 60 
                         ray_num = 64, # number of steering angles in the range prescribed by sweep 
                         imaging_ndims = 2, # only 2D plane wave supported right now
                         )
                for i in range(n_transducers)]

for t in transducers:
    t.make_sensor_coords(phantom.baseline[0])

transducer_set = TransducerSet(transducers, seed=8888)
transducer_set.generate_extrinsics(shape="spherical", extrinsics_kwargs={'r_mean': 50e-3, 'view_std': 0, 'yaw_fraction': 0.3, 'pitch_fraction': 0.6, 'roll_fraction': 0})

transducer_set.plot_transducer_fovs(scale=0.05)
transducer_set.plot_transducers()

sensor_nl = Sensor(transducer_set=transducer_set, aperture_type="single_transducer")
sensor_extended = Sensor(transducer_set=transducer_set, aperture_type="extended")

simprops = SimProperties(grid_size   = (1024,128,128),
                 PML_size    = (32,8,8),
                 PML_alpha   = 2,
                 t_end       = 12e-5,           # [s]
                 bona        = 6,               # parameter b/a determining degree of nonlinear acoustic effects
                 alpha_coeff = 0.5, 	        # [dB/(MHz^y cm)]
                 alpha_power = 1.5,
                 )

nl_experiment = Experiment(simulation_path = '../pw_single_aperture_01',
                           sim_properties  = simprops,
                           phantom         = phantom,
                           transducer_set  = transducer_set,
                           sensor          = sensor_nl,
                           nodes           = 2,
                           results         = None,
                           indices         = None,
                           workers         = 3,
                           )

extended_experiment = Experiment(simulation_path = '../pw_extended_aperture_01',
                                 sim_properties  = simprops,
                                 phantom         = phantom,
                                 transducer_set  = transducer_set,
                                 sensor          = sensor_extended,
                                 nodes           = 2,
                                 results         = None,
                                 indices         = None,
                                 workers         = 3,
                                 )

nl_experiment.save()
nl_experiment.run(dry=True)
nl_experiment.add_results()
#nl_reconstruction = Compounding(nl_experiment) # beta, may not currently work as intended

extended_experiment.save()
extended_experiment.run(dry=True)
extended_experiment.add_results()
#extended_reconstruction = Compounding(extended_experiment)



